In [162]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

# nltk.download('stopwords')
# nltk.download('wordnet')

# Multinomial Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB

# Import Tf-idf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Enable Jupyter Notebook's intellisense
%config IPCompleter.greedy=True

# Import the Label Encoder
from sklearn.preprocessing import LabelEncoder

# Import the train test split
from sklearn.model_selection import train_test_split

In [163]:
import pandas as pd

data = pd.read_csv(r"C:\Users\Simrun Sharma\Desktop\NLP\parsed_rows_head.csv")
print(data.sample(6))

    Record ID  Smoking Status  \
1         643  CURRENT SMOKER   
8         412      NON-SMOKER   
12        416         UNKNOWN   
13        417         UNKNOWN   
5         590         UNKNOWN   
7         542      NON-SMOKER   

                                                 Text  
1   026738007\nCMC\n15319689\n3/25/1998 12:00:00 A...  
8   194442600 RWH\n1642008\n83328/rh01\n61509\n10/...  
12  517502848 ELMVH\n18587541\n6634152\n12/12/2004...  
13  895872725 ELMVH\n99080881\n979718\n5/25/2002 1...  
5   463067067\nFIH\n7339944\n31403/p8v6\n363828\n1...  
7   407060601\nFIH\n9835577\n74261/1069\n663014\n0...  


In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Record ID       16 non-null     int64 
 1   Smoking Status  16 non-null     object
 2   Text            16 non-null     object
dtypes: int64(1), object(2)
memory usage: 516.0+ bytes


In [165]:
print(data["Smoking Status"].value_counts())

UNKNOWN           7
CURRENT SMOKER    5
NON-SMOKER        2
PAST SMOKER       2
Name: Smoking Status, dtype: int64


In [166]:
le = LabelEncoder()
data["Smoking_enc"] = le.fit_transform(data["Smoking Status"])
display(data.sample(6))

,Record ID,Smoking Status,Text,Smoking_enc
0,641,CURRENT SMOKER,977146916\nHLGMC\n2878891\n022690\n01/27/1997 ...,0
9,623,PAST SMOKER,512875472\nPRGH\n47241971\n271075\n4/26/1999 1...,2
15,736,PAST SMOKER,337483886\nCTMC\n59100048\n219781\n03/30/1993 ...,2
3,704,CURRENT SMOKER,418520250\nNVH\n61562872\n3/11/1995 12:00:00 A...,0
13,417,UNKNOWN,895872725 ELMVH\n99080881\n979718\n5/25/2002 1...,3
6,735,UNKNOWN,538617688\nPUOMC\n7271129\n234750\n118602\n4/2...,3


In [167]:
# Python
data["Smoking_enc"] = data["Smoking_enc"].astype("int64")
print(data.dtypes)

Record ID          int64
Smoking Status    object
Text              object
Smoking_enc        int64
dtype: object


In [168]:
data["word_count"] = data["Text"].str.split().str.len()
print(data.groupby("Smoking Status")["word_count"].mean())

Smoking Status
CURRENT SMOKER    1016.200000
NON-SMOKER        1133.000000
PAST SMOKER       1326.000000
UNKNOWN            488.714286
Name: word_count, dtype: float64


In [169]:
print(data["Text"])

0     977146916\nHLGMC\n2878891\n022690\n01/27/1997 ...
1     026738007\nCMC\n15319689\n3/25/1998 12:00:00 A...
2     071962960\nBH\n4236518\n417454\n12/10/2001 12:...
3     418520250\nNVH\n61562872\n3/11/1995 12:00:00 A...
4     301443520\nCTMC\n49020928\n448922\n1/11/1990 1...
5     463067067\nFIH\n7339944\n31403/p8v6\n363828\n1...
6     538617688\nPUOMC\n7271129\n234750\n118602\n4/2...
7     407060601\nFIH\n9835577\n74261/1069\n663014\n0...
8     194442600 RWH\n1642008\n83328/rh01\n61509\n10/...
9     512875472\nPRGH\n47241971\n271075\n4/26/1999 1...
10    917989835 RWH\n5427551\n405831\n9660879\n01/09...
11    817406016 RWH\n3154334\n554691\n3547577\n7/6/2...
12    517502848 ELMVH\n18587541\n6634152\n12/12/2004...
13    895872725 ELMVH\n99080881\n979718\n5/25/2002 1...
14    305757070 ELMVH\n74973293\n2973118\n11/27/2003...
15    337483886\nCTMC\n59100048\n219781\n03/30/1993 ...
Name: Text, dtype: object


In [170]:
import re

clean_messages = data["Text"].str.lower()
clean_text = []
for message in clean_messages:
    pattern = r"(\S+\s){0,15}\S*(smok|tobacco|cigar|pack|ppd)\S*(\s\S+){0,20}"
    match = re.search(pattern, message, re.IGNORECASE)

    if match:
        matched_text = match.group(0)
        clean_text.append(matched_text)
    else:
        clean_text.append(message)

data["Text"] = clean_text
print(data["Text"])

0     social history :\nlives in merca .\ndrinks gin...
1     next-door .\nshe reports having no siblings .\...
2     ) .\npatient denies current use .\npatient rep...
3     cm .\nallergies :\nshellfish , which causes na...
4     causes hives .\nsocial history :\nshe drinks a...
5     463067067\nfih\n7339944\n31403/p8v6\n363828\n1...
6     538617688\npuomc\n7271129\n234750\n118602\n4/2...
7     .\nallergies :\nthere were no known drug aller...
8     for further management .\npast medical history...
9     :\nshe is allergic to quinine sulfate .\nsocia...
10    917989835 rwh\n5427551\n405831\n9660879\n01/09...
11    817406016 rwh\n3154334\n554691\n3547577\n7/6/2...
12    nkda\npreop hct 36.4 3/16/04 pod 0 s / p a / p...
13    895872725 elmvh\n99080881\n979718\n5/25/2002 1...
14    305757070 elmvh\n74973293\n2973118\n11/27/2003...
15    two blocks .\nhe has a known tuberculosis and ...
Name: Text, dtype: object


In [171]:
tokenized_messages = data["Text"].str.lower().apply(word_tokenize)
print(tokenized_messages)

0     [social, history, :, lives, in, merca, ., drin...
1     [next-door, ., she, reports, having, no, sibli...
2     [), ., patient, denies, current, use, ., patie...
3     [cm, ., allergies, :, shellfish, ,, which, cau...
4     [causes, hives, ., social, history, :, she, dr...
5     [463067067, fih, 7339944, 31403/p8v6, 363828, ...
6     [538617688, puomc, 7271129, 234750, 118602, 4/...
7     [., allergies, :, there, were, no, known, drug...
8     [for, further, management, ., past, medical, h...
9     [:, she, is, allergic, to, quinine, sulfate, ....
10    [917989835, rwh, 5427551, 405831, 9660879, 01/...
11    [817406016, rwh, 3154334, 554691, 3547577, 7/6...
12    [nkda, preop, hct, 36.4, 3/16/04, pod, 0, s, /...
13    [895872725, elmvh, 99080881, 979718, 5/25/2002...
14    [305757070, elmvh, 74973293, 2973118, 11/27/20...
15    [two, blocks, ., he, has, a, known, tuberculos...
Name: Text, dtype: object


In [172]:
# def alpha(tokens):
#     """This function removes all non-alphanumeric characters"""
#     alpha = []
#     for token in tokens:
#         if str.isalpha(token):
#             alpha.append(token)
#     return alpha


# # Apply our function to tokens
# tokenized_messages = tokenized_messages.apply(alpha)

# print(tokenized_messages)

In [173]:
# import re
# def clean(tokens):
#     clean_text = []
#     for token in tokens:
#         if str.__contains__
#         pattern = r"(\S+\s){0,10}\S*(smok|tobacco|cigar|pack|ppd)\S*(\s\S+){0,10}"
#         match = re.search(pattern, token, re.IGNORECASE)
#         # print(match)

#         if match:
#             # Extract the matched part
#             matched_text = match.group(0)
#             # print(matched_text)
#             clean_text.append(matched_text)
#             # print(matched_text)
#         # else:
#         #     clean_text.append(token)
#     # print(clean_text)
#     return clean_text

# tokenized_messages = tokenized_messages.apply(clean)

# print(tokenized_messages)

In [174]:
# def remove_stop_words(tokens):
#     """This function removes all stop words in terms of nltk stopwords"""
#     no_stop = []
#     for token in tokens:
#         if token not in stopwords.words('english'):
#             no_stop.append(token)
#     return no_stop

# # Apply our function to tokens
# tokenized_messages = tokenized_messages.apply(remove_stop_words)

# print(tokenized_messages)

In [175]:
# def lemmatize(tokens):
#     """This function lemmatize the messages"""
#     # Initialize the WordNetLemmatizer
#     lemmatizer = WordNetLemmatizer()
#     # Create the lemmatized list
#     lemmatized = []
#     for token in tokens:
#             # Lemmatize and append
#             lemmatized.append(lemmatizer.lemmatize(token))
#     return " ".join(lemmatized)

# # Apply our function to tokens
# # tokenized_messages = tokenized_messages.apply(lemmatize)

# print(tokenized_messages)

0     [social, history, :, lives, in, merca, ., drin...
1     [next-door, ., she, reports, having, no, sibli...
2     [), ., patient, denies, current, use, ., patie...
3     [cm, ., allergies, :, shellfish, ,, which, cau...
4     [causes, hives, ., social, history, :, she, dr...
5     [463067067, fih, 7339944, 31403/p8v6, 363828, ...
6     [538617688, puomc, 7271129, 234750, 118602, 4/...
7     [., allergies, :, there, were, no, known, drug...
8     [for, further, management, ., past, medical, h...
9     [:, she, is, allergic, to, quinine, sulfate, ....
10    [917989835, rwh, 5427551, 405831, 9660879, 01/...
11    [817406016, rwh, 3154334, 554691, 3547577, 7/6...
12    [nkda, preop, hct, 36.4, 3/16/04, pod, 0, s, /...
13    [895872725, elmvh, 99080881, 979718, 5/25/2002...
14    [305757070, elmvh, 74973293, 2973118, 11/27/20...
15    [two, blocks, ., he, has, a, known, tuberculos...
Name: Text, dtype: object


In [176]:
data["Text"] = tokenized_messages
data.sample(6)

,Record ID,Smoking Status,Text,Smoking_enc,word_count
5,590,UNKNOWN,"[463067067, fih, 7339944, 31403/p8v6, 363828, ...",3,923
0,641,CURRENT SMOKER,"[social, history, :, lives, in, merca, ., drin...",0,700
11,403,UNKNOWN,"[817406016, rwh, 3154334, 554691, 3547577, 7/6...",3,151
4,757,CURRENT SMOKER,"[causes, hives, ., social, history, :, she, dr...",0,546
3,704,CURRENT SMOKER,"[cm, ., allergies, :, shellfish, ,, which, cau...",0,1569
1,643,CURRENT SMOKER,"[next-door, ., she, reports, having, no, sibli...",0,869


In [177]:
# Select the features and the target
X = data["Text"]
y = data["Smoking_enc"]

In [178]:
text_view = data["Text"]

In [179]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=34, stratify=y
)

X_train = X_train.astype(str)
X = X.astype(str)
X_test = X_test.astype(str)

In [186]:
# Create the tf-idf vectorizer
vectorizer = TfidfVectorizer(strip_accents="ascii")


# First fit the vectorizer with our training set
tfidf_train = vectorizer.fit_transform(X_train)
print(tfidf_train)

# Now we can fit our test data with the same vectorizer
tfidf_test = vectorizer.transform(X_test)

  (0, 588)	0.12747579235279002
  (0, 137)	0.22611265748244022
  (0, 497)	0.22611265748244022
  (0, 168)	0.1941881768033229
  (0, 914)	0.13961291661184044
  (0, 396)	0.22611265748244022
  (0, 195)	0.22611265748244022
  (0, 138)	0.1941881768033229
  (0, 251)	0.22611265748244022
  (0, 877)	0.1941881768033229
  (0, 622)	0.11696213709947532
  (0, 838)	0.12747579235279002
  (0, 253)	0.15396808774953172
  (0, 421)	0.1715373972909578
  (0, 274)	0.1941881768033229
  (0, 614)	0.1941881768033229
  (0, 907)	0.22611265748244022
  (0, 625)	0.3430747945819156
  (0, 850)	0.22611265748244022
  (0, 113)	0.15396808774953172
  (0, 310)	0.1941881768033229
  (0, 768)	0.15396808774953172
  (0, 415)	0.11696213709947532
  (0, 790)	0.1715373972909578
  (0, 416)	0.22611265748244022
  :	:
  (11, 467)	0.18708854427342433
  (11, 787)	0.18708854427342433
  (11, 775)	0.18708854427342433
  (11, 403)	0.37417708854684867
  (11, 727)	0.37417708854684867
  (11, 307)	0.18708854427342433
  (11, 560)	0.18708854427342433
  (1

In [184]:
# Initialize the Multinomial Naive Bayes classifier
nb = MultinomialNB()

# Fit the model
nb.fit(tfidf_train, y_train)

# Print the accuracy score
print("Accuracy:", nb.score(tfidf_test, y_test))

Accuracy: 0.75


In [182]:
# y_pred = nb.predict(tfidf_test)

# # Print the Confusion Matrix
# cm = confusion_matrix(y_test, y_pred)
# print("Confusion Matrix\n")
# print(cm)

In [183]:
import re

clean_text = []
for token in tokenized_messages:
    print(token)
#         pattern = r"(\S+\s){0,10}\S*(smok|tobacco|cigar|pack|ppd)\S*(\s\S+){0,10}"
#         match = re.search(pattern, token, re.IGNORECASE)


#         if match:
#             # Extract the matched part
#             matched_text = match.group(0)
#             clean_text.append(matched_text)

# print(clean_text)

['social', 'history', ':', 'lives', 'in', 'merca', '.', 'drinks', 'ginger', 'brandy', 'to', 'excess', ',', 'pipe', 'and', 'cigar', 'smoker', 'for', 'many', 'years', '.', 'physical', 'examination', ':', 'in', 'general', 'was', 'a', 'cachectic', ',', 'jaundiced', 'man', '.', 'bloodpressure', ':', '124/60']
['next-door', '.', 'she', 'reports', 'having', 'no', 'siblings', '.', 'habits', ':', 'the', 'patient', 'has', 'a', '20', 'pack-year', 'smoking', 'history', '.', 'she', 'reports', 'having', 'just', 'quit', 'prior', 'to', 'admission', '.', 'she', 'denies', 'any', 'alcohol', 'or', 'street', 'drug', 'use']
[')', '.', 'patient', 'denies', 'current', 'use', '.', 'patient', 'reports', 'occasional', 'alcohol', 'use', '.', 'has', 'been', 'smoking', 'approximately', '10', 'cigarettes', 'a', 'day', '.', 'claims', 'to', 'have', 'stopped', 'a', '``', 'few', 'weeks', 'ago', '``', '.', 'married', 'with', 'two']
['cm', '.', 'allergies', ':', 'shellfish', ',', 'which', 'causes', 'nausea', 'and', 'vomit